In [2]:
import csv
import os
import random
import numpy as np
import pandas as pd
from traces_dataset import Traces_Dataset
import ast
import torch 
from MLP_model import Net

In [3]:
dataset = Traces_Dataset('dataset_test.csv')

In [4]:
exp_narva_df = pd.read_csv('experiment records narval new.csv')

In [5]:
normalized_mean = torch.tensor(ast.literal_eval(exp_narva_df['normalize_mean'][15]))
normalized_std = torch.tensor(ast.literal_eval(exp_narva_df['normalize_std'][15]))

In [6]:
dataset.clean_features()

In [7]:
dataset.normalize_test(normalized_mean, normalized_std)

In [8]:
target_params = dataset.params

In [9]:
import torch

# Specify the path to your saved model
model_path = exp_narva_df['model'][15]

# Load the model with map_location
device = torch.device('cpu')  # Map tensors to CPU
model = torch.load(model_path, map_location=device)

In [10]:
model = Net(dataset.inputs.shape[1])

In [11]:
model.load_state_dict(torch.load(exp_narva_df['model'][15], map_location=device))

<All keys matched successfully>

In [12]:
estimated_params = model(dataset.inputs)

In [42]:
estimated_params.shape, target_params.shape

(torch.Size([1000, 8]), torch.Size([1000, 8]))

In [15]:
mse_each_param = (estimated_params - target_params)**2
print(mse_each_param)

tensor([[4.8016e+00, 2.2343e+00, 3.1916e+00,  ..., 8.5660e-01, 9.9321e-01,
         2.2983e-02],
        [6.6767e-02, 3.2031e-01, 1.8418e-02,  ..., 7.6059e-04, 3.6957e-01,
         2.3242e-02],
        [1.8740e+00, 2.4259e-01, 2.5492e-03,  ..., 2.0309e+00, 3.1924e-01,
         4.7253e-01],
        ...,
        [4.0961e+01, 5.9379e+01, 1.8904e-02,  ..., 4.5999e+00, 1.3634e-01,
         3.2624e-01],
        [4.7879e-01, 3.6489e-01, 1.0378e-02,  ..., 6.8852e+00, 1.4496e-01,
         1.1568e+00],
        [9.5371e+00, 2.3254e+01, 5.3095e-02,  ..., 5.9517e-02, 2.9019e-01,
         8.8415e-02]], grad_fn=<PowBackward0>)


In [19]:
avg_mse_each_param = torch.mean((estimated_params - target_params)**2, dim=0)
print(avg_mse_each_param)

tensor([12.9173, 10.4419,  0.5615, 10.0132,  2.8792, 12.5059,  1.1472,  2.0171],
       grad_fn=<MeanBackward1>)


In [20]:
overall_mse = torch.mean(avg_mse_each_param)
print(overall_mse)

tensor(6.5604, grad_fn=<MeanBackward0>)


In [21]:
avg_re_each_param = torch.mean(((estimated_params - target_params)/target_params)**2, dim=0)
print(avg_re_each_param)

tensor([9.8675e-04, 1.2993e-03, 5.0390e-02, 2.8947e+01, 1.2528e-01, 5.2929e-01,
        5.1218e-02, 8.3912e-02], grad_fn=<MeanBackward1>)


In [22]:
overall_re = torch.mean(avg_re_each_param)
print(overall_re)

tensor(3.7236, grad_fn=<MeanBackward0>)


In [24]:
dataset = Traces_Dataset('dataset_test.csv')

In [27]:
dataset.time_traces.shape, dataset.current_traces.shape, dataset.params.shape

(torch.Size([1000, 12, 20]), torch.Size([1000, 12, 20]), torch.Size([1000, 8]))

In [34]:
target_time_traces_set = dataset.time_traces.numpy()
target_current_traces_set = dataset.current_traces.numpy()
target_params_set = dataset.params.numpy()

In [35]:
target_time_traces_set.shape, target_current_traces_set.shape, target_params_set.shape

((1000, 12, 20), (1000, 12, 20), (1000, 8))

In [36]:
from GA_test import ga_algorithm

In [37]:
ga_algorithm(target_time_traces_set[0], target_current_traces_set[0], target_params_set[0])

Experiment ID: 22815183
1th Optimum Solution:  [128.22188764499822, -80.02136029223942, 36.636389882703156, 34.73110086983161, -38.28934151410932, 43.544789962649475, -44.82165837459445, 39.165527182315365]
{'Min': 14608306.280788608, 'Max': 29645417661.39089, 'Avg': 803137931.6531272, 'Std': 1298704768.725753, 'Generation': 1}
{'Min': 5179353.678392287, 'Max': 32922083120.744335, 'Avg': 382638931.98117965, 'Std': 1123437950.9286318, 'Generation': 2}
{'Min': 2792982.7139410316, 'Max': 69725255265.81367, 'Avg': 308764780.97681993, 'Std': 1788394762.9741168, 'Generation': 3}
{'Min': 1478584.0360958846, 'Max': 12583472387.851852, 'Avg': 228221893.29075512, 'Std': 741793629.6460779, 'Generation': 4}
{'Min': 1411177.592229811, 'Max': 20177707687.81855, 'Avg': 258264754.60586652, 'Std': 991810908.598639, 'Generation': 5}
{'Min': 724412.6473952099, 'Max': 15615046264.435219, 'Avg': 220929756.58078924, 'Std': 786686234.7707369, 'Generation': 6}
{'Min': 724412.6473952099, 'Max': 7781531701.5278

KeyboardInterrupt: 